In this project, I am working on advanced lane finding

## Table of Contents

Here I am showing the structure of the notebook and also acknowledging the sources I took help from, for each section

1. [Camera Calibration](#camera_calibration) - ***./examples/example.ipynb***
2. [Undistort Image](#undistort_image) - ***lesson 6 Part 12 quiz solution***
3. [Perspective Transform](#perspective_transform) - ***Jeremy shannon project [here](https://github.com/jeremy-shannon/CarND-Advanced-Lane-Lines/blob/master/project.ipynb)***
4. [Convert RGB image into Binary](#convert_to_binary) - ***lesson 7 part 12 quiz solution***
5. [Finding Lane Pixels](#finding_lane_pixels) - ***Lesson 8 Part 4 quiz solution***
6. [Radius Of Curvature](#radius_of_curvature) - ***Lesson 8 part 7 quiz solution***
7. [Testing on Test Images](#testing_on_test_images) -  ***putting text code block was from jeremy shannons blog***
8. [Video Pipeline](#video_pipeline) - ***putting text code block was from jeremy shannons blog***
9. [Search from Prior](#search_prior) ***Lesson 8 Part 5 quiz solution***

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from camera import Camera
from preprocess import PerspectiveTransformer
import util
import os


In [2]:
from lane import LaneDetector
from util import 

ModuleNotFoundError: No module named 'lane_utils'

In [3]:
import tqdm

In [ ]:
ld = LaneDetector()

In [ ]:
%matplotlib inline

In [ ]:
def all_transform(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = pt.transform(img)
    img = bt.transform(img)
    return img

<a id="camera_calibration">
</a>

## Camera Calibration

In this section , I am writing code to calibrate camera. I have used codes from the ```example.ipynb``` provided with this repository. 

At first, we are calculating the calibration matrices. 

In [ ]:
H = 720
W = 1280
IMG_SIZE = (H,W)

In [ ]:
camera_cal_output_dir = "result/camera_cal/"
os.makedirs(camera_cal_output_dir, exist_ok=True)

undistort_output_dir = "result/undistort/"
os.makedirs(undistort_output_dir, exist_ok=True)

In [ ]:
camera = Camera(IMG_SIZE)

In [ ]:
imgs = util.load_img_from_dir("camera_cal")


In [ ]:
for i, img in enumerate(imgs):
    camera.set_img_points(img, plot=True, save_fig=True, fig_name=os.path.join(camera_cal_output_dir,"{}.jpg".format(i)))
    

The calibration matrix

In [ ]:
calibration_matrix, _ = camera.get_calibration_matrix()

In [ ]:

print(calibration_matrix)

### Resulting images

The undistorted images. Note: not all undistortions are noticeable as not all distortions can be noticed easily

In [ ]:
imgs = util.load_img_from_dir('camera_cal/')

In [ ]:
for i, img in tqdm.tqdm(enumerate(imgs)):
    undist = camera.undistort(img)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image')
    ax2.imshow(undist)
    ax2.set_title('Undistorted Image')
    plt.savefig(os.path.join(undistort_output_dir, "image_{}".format(i)))

Following images undistortion is clearly noticeable

![undistorted4](undistort_result/camera_cal/calibration1.jpg)

In this following image, it is not that noticeable

![undistorted3](undistort_result/camera_cal/calibration10.jpg)

<a id="undistort_image"></a>
### Undistort Image

I took help from the ***lesson 6 Part 12 quiz solution***

Lets read an image

In [ ]:
image_name = 'test_images/test2.jpg'

In [ ]:
img = util.imread(image_name)

Undistort it using the calibration matrix

In [ ]:
img = camera.undistort(img)

Let's have a look

In [ ]:
plt.imshow(img)

Saving the image

In [ ]:
cv2.imwrite('result/undistorted_test_image.jpg',img)

<a id="perspective_transform"></a>
## Perspective Transform

Lets plot the image in a grid for getting the four points for the perspective transform. I took help from the project done by Jeremy shannon which is referenced [here](https://github.com/jeremy-shannon/CarND-Advanced-Lane-Lines/blob/master/project.ipynb)

In [ ]:
pt = PerspectiveTransformer()

In [ ]:
pt.tune(img)

In [ ]:
img_transformed = pt.transform(img)

In [ ]:
plt.imshow(img_transformed)

Perfect!!

In [ ]:
cv2.imwrite('result/image_transformed.jpg',img_transformed)


<a id ="convert_to_binary"></a>
## Convert to Binary

In [ ]:
from preprocess import BinaryTransformer

In [ ]:
bt = BinaryTransformer()

In [ ]:
test_imgs = [pt.transform(img) for img in util.load_img_from_dir('test_images')]


In [ ]:
binary_config_file = "config.yaml"
if not os.path.exists(binary_config_file):
    bt.tune_imgs(test_imgs)
else:
    bt.load_config(binary_config_file)

In [ ]:
print(bt.s_thresh)

In [ ]:
print(bt.sobel_thresh)

In [ ]:
binary = bt.r2b(img_transformed)

plotting the actual image and binary. This is also taken from lesson 7 part 11

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2)
f.tight_layout()
ax1.imshow(img_transformed)
ax1.set_title('Original Image')
ax2.imshow(binary, cmap='gray')
ax2.set_title('Binary')

saving the binary image

In [ ]:
cv2.imwrite('result/binary.jpg',binary)

<a id="finding_lane_pixels"></a>
### Finding Lane pixels

The function to find out pixels of a lane. I took help from the ***Lesson 8 Part 4*** quiz solution 

out_img,left_fitx,right_fitx,ploty,left_fit,right_fit = ld.fit(binary)
plt.plot(left_fitx, ploty, color='red')
plt.plot(right_fitx, ploty, color='blue')
plt.imshow(img_transformed)
plt.title('Lanes')
plt.savefig('result/image_transformed_with_lane.jpg')

### Let's draw the lanes in the original picture!

***I took help from jeremy shannon's [project](https://github.com/jeremy-shannon/CarND-Advanced-Lane-Lines/blob/master/project.ipynb) for this section***

In [ ]:
image_lane = ld.draw_lane(img,binary,left_fitx,right_fitx,ploty,pt)

In [ ]:
plt.imshow(image_lane)
plt.savefig('result/image_lane.jpg')

<a id ="radius_of_curvature"></a>
## Radius of curvature

Measuring the radius of curvature. I took help from ***Lesson 8 part 7 quiz solution*** for this section

In [ ]:
left_curverad,right_curverad,vehicle_position = measure_curvature_pos(ploty, left_fitx, right_fitx, binary)

The curvature radius is the average of the left lane and right lane radius

In [ ]:
curv_radius = (left_curverad+right_curverad)/2


In [ ]:
text="Curvature radius {} m , and vehicle position w.r.t lane midpoint {} m".format(curv_radius,vehicle_position)

In [ ]:
text="Curve radius {:04.2f} m".format(curv_radius)
cv2.putText(image_lane, text, (50,70), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
text="vehicle position w.r.t center {:04.2f} m".format(vehicle_position)
image_lane = cv2.putText(image_lane, text, (50,100), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 2, cv2.LINE_AA)

In [ ]:

plt.imshow(image_lane)

In [ ]:
cv2.imwrite('result/detected_lane_with_radius_position.jpg',image_lane)

<a id="testing_on_test_images"></a>
## Testing on the test images

In [ ]:
test_images = glob.glob('test_images/*.jpg')

In [ ]:
for test_image_name in test_images:
    test_image = util.imread(test_image_name)
    test_image_transformed = pt.transform(test_image)
    binary = bt.r2b(test_image_transformed)
    out_img,left_fitx,right_fitx,ploty,left_fit,right_fit = ld.fit(binary)
    test_image_lane = ld.draw_lane(test_image,binary,left_fitx,right_fitx,ploty,pt)
    left_curverad,right_curverad,vehicle_position = measure_curvature_pos(ploty,left_fitx,right_fitx,binary) 
    curv_radius = (left_curverad+right_curverad) / 2
    
    text="Curve radius {:04.2f} m".format(curv_radius)
    cv2.putText(test_image_lane, text, (50,70), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    text="vehicle position w.r.t center {:04.2f} m".format(vehicle_position)
    cv2.putText(test_image_lane, text, (50,100), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    
    f, (ax1, ax2, ax3,ax4) = plt.subplots(1, 4, figsize=(24,9))
    f.tight_layout()
    ax1.imshow(test_image)
    ax1.set_title('Original Image')
    ax2.imshow(test_image_transformed)
    ax2.set_title('Transformed Image')
    ax3.imshow(out_img, cmap='gray')
    ax3.set_title('Binary')
    ax4.imshow(test_image_lane, cmap='gray')
    ax4.set_title('Detected Lane')


<a id="video_pipeline"></a>
## Video Pipeline

Let's read the video


<a id="search_prior"></a>
### But wait!

From ***lesson 8 Part 5*** we learned that not all pixels in every frame is necessary to search the lane. It is enough to search around the previously detected lanes. so at first, let's get the polynomial coefficients for the first frame

In [ ]:
def prior_search(binary_frame,left_fit,right_fit,margin=50):
    nonzero = binary_frame.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    left_lane_indices = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_indices = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    leftx = nonzerox[left_lane_indices]
    lefty = nonzeroy[left_lane_indices]
    rightx = nonzerox[right_lane_indices]
    righty = nonzeroy[right_lane_indices]
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, binary_frame.shape[0]-1, binary_frame.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fitx,right_fitx,ploty,left_fit,right_fit
    

Lets close the video object and read it again!

In [ ]:
cap = cv2.VideoCapture('project_video.mp4')

Lets take the width and height of the video to create the ```VideoWriter``` object for output of the video

I took help for the following code from this [link](https://www.learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/)

In [ ]:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('project_output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

Importing display and widgets module for display

Let's get the coefficients for the first frame

In [ ]:
ret,frame = cap.read()
binary_frame = all_transform(frame)
frame_output,left_fitx,right_fitx,ploty,left_fit,right_fit = fit(binary_frame)

### Line Class for tracking

As suggested in the lessons, I am declaring the line class to track the lanes

In [ ]:
left_line = Line()
right_line= Line()

In [ ]:
while ret:    
    if left_line.detected ==False or right_line.detected==False:
        frame_output,left_fitx,right_fitx,ploty,left_fit,right_fit = ld.fit(binary_frame)    
    else:
        left_fitx,right_fitx,ploty,left_fit,right_fit = prior_search(binary_frame,left_fit,right_fit,margin=150)
    if abs(abs(left_fitx[-1]-right_fitx[-1])-500)<100:
        left_line.detected=False
        right_line.detected=False
        left_line.fit(left_fit)
        right_line.fit(right_fit)
        left_best_fitx=left_line.get_fitx(ploty)
        right_best_fitx=right_line.get_fitx(ploty)
    
    frame_lane = draw_lane(frame,binary_frame,left_best_fitx,right_best_fitx,ploty, pt)
    left_curverad,right_curverad,vehicle_position=measure_curvature_pos(ploty,left_best_fitx,right_best_fitx,binary_frame) 
    curv_radius = (left_curverad+right_curverad)/2
    text="Curve radius {:04.2f} m".format(curv_radius)
    cv2.putText(frame_lane, text, (50,70), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    text="vehicle position w.r.t center {:04.2f} m".format(vehicle_position)
    cv2.putText(frame_lane, text, (50,100), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    out.write(frame_lane)
    diff = left_curverad-right_curverad
    ret, frame = cap.read()
    if ret:
        binary_frame = all_transform(frame)
    
cap.release()
out.release()